In [ ]:
# import numpy as np
# from scipy.stats import binom

# def sequential_massart_absolute(n_samples, p_threshold, epsilon, delta):
#     """
#     Sequential Massart Algorithm for absolute error.

#     Parameters:
#     - n_samples: int, the number of samples to draw.
#     - p_threshold: float, the threshold probability to test against.
#     - epsilon: float, the absolute error tolerance.
#     - delta: float, the confidence level (1 - delta).

#     Returns:
#     - estimate: float, the estimated probability.
#     - accepted: bool, whether the hypothesis is accepted.
#     """
#     successes = 0
#     for t in range(1, n_samples + 1):
#         # Draw a sample (Bernoulli trial)
#         sample = np.random.binomial(1, p_threshold)
#         successes += sample

#         # Estimate probability
#         p_hat = successes / t

#         # Calculate the upper and lower bounds
#         lower_bound = p_hat - epsilon
#         upper_bound = p_hat + epsilon

#         # Check if the true probability is within the bounds
#         if lower_bound <= p_threshold <= upper_bound:
#             return p_hat, True

#     return p_hat, False

# def sequential_massart_relative(n_samples, p_threshold, epsilon_r, delta):
#     """
#     Sequential Massart Algorithm for relative error.

#     Parameters:
#     - n_samples: int, the number of samples to draw.
#     - p_threshold: float, the threshold probability to test against.
#     - epsilon_r: float, the relative error tolerance.
#     - delta: float, the confidence level (1 - delta).

#     Returns:
#     - estimate: float, the estimated probability.
#     - accepted: bool, whether the hypothesis is accepted.
#     """
#     successes = 0
#     for t in range(1, n_samples + 1):
#         # Draw a sample (Bernoulli trial)
#         sample = np.random.binomial(1, p_threshold)
#         successes += sample

#         # Estimate probability
#         p_hat = successes / t

#         # Calculate the relative error bounds
#         lower_bound = p_hat * (1 - epsilon_r)
#         upper_bound = p_hat * (1 + epsilon_r)

#         # Check if the true probability is within the bounds
#         if lower_bound <= p_threshold <= upper_bound:
#             return p_hat, True

#     return p_hat, False

# # Example usage
# n_samples = 1000
# p_threshold = 0.5
# epsilon_absolute = 0.05
# epsilon_relative = 0.1
# delta = 0.01

# # Absolute error test
# estimate_abs, accepted_abs = sequential_massart_absolute(n_samples, p_threshold, epsilon_absolute, delta)
# print(f"Absolute Error: Estimate = {estimate_abs}, Accepted = {accepted_abs}")

# # Relative error test
# estimate_rel, accepted_rel = sequential_massart_relative(n_samples, p_threshold, epsilon_relative, delta)
# print(f"Relative Error: Estimate = {estimate_rel}, Accepted = {accepted_rel}")


In [ ]:
from abc import ABC, abstractmethod
from scipy.stats import beta

class ConfidenceInterval(ABC):

    @abstractmethod
    def __init__(self, func):
        self.func = func

    def __call__(self, *args):
        return self.func(*args)


class CPInterval(ConfidenceInterval):
    def __init__(self):
        super().__init__(self.func)

    def func(self, alpha, n, m):
        a = beta.ppf(alpha/2, m, n-m+1)
        b = beta.ppf(1-alpha/2, m+1, n-m)

        return a, b

In [ ]:
import numpy as np

def h(gamma, epsilon):
    if gamma < 0.5:
        return 4.5 / ((3*gamma + epsilon)*(3*(1-gamma) - epsilon))
    return 4.5 / ((3*(1-gamma) + epsilon)*(3*gamma + epsilon))

def sequential_massart_absolute(epsilon, delta, alpha, samples, interval=CPInterval()):

    M = np.ceil(1 / (2 * epsilon**2) * np.log(2 / delta))
    n = M
    m = 0

    a = 0
    b = 1

    for k, sample in enumerate(samples):
        k += 1
        m += sample
        # m += 5
        a, b = interval(alpha, k, m)

        print(f'{k}: Interval: [{a}, {b}] - Samples: {n} - Estimate: {m/(k)}')

        if a <= 0.5 <= b:
            n = M
        elif b < 0.5:
            n = np.ceil(2 / (h(b, epsilon)*epsilon**2) * np.log(2 / (delta - alpha)))

            if n > (h(b, epsilon)*epsilon**2) * np.log(2 / (delta - alpha)):
                return m / k, True
        else:
            n = np.ceil(2 / (h(a, epsilon)*epsilon**2) * np.log(2 / (delta - alpha)))

            if n > (h(a, epsilon)*epsilon**2) * np.log(2 / (delta - alpha)):
                return m / k, True

        n = min(n, M)
        if n <= k:
            return m / k, True

    return m / k, False

In [ ]:
import numpy as np

# Parameters
mean = 0.90
std_dev = 0.0010000

# Generate random samples
samples = np.random.normal(loc=mean, scale=std_dev, size=200) # size specifies the number of samples

# print(samples[:20])

sequential_massart_absolute(epsilon=0.01, delta=0.05, alpha=0.001, samples=samples)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta

# Parameters for the Beta distribution
u = 2  # shape parameter alpha
v = 5  # shape parameter beta
alpha = 0.05  # quantile we are interested in

# Generate values for the x-axis
x = np.linspace(0, 1, 1000)

# Calculate the Beta distribution PDF
pdf = beta.pdf(x, u, v)

# Calculate the CDF values
cdf = beta.cdf(x, u, v)

# Calculate the alpha-th quantile
quantile = beta.ppf(alpha, u, v)

print(f"The {alpha}-th quantile for Beta({u}, {v}) is {quantile:.4f}")

# Plot the Beta distribution PDF and CDF
plt.figure(figsize=(14, 6))

# Subplot 1: PDF
plt.subplot(2, 1, 1)
plt.plot(x, pdf, label=f'Beta({u}, {v}) PDF')
plt.axvline(quantile, color='red', linestyle='--', label=f'{alpha}-th quantile')
plt.title('Beta Distribution PDF')
plt.ylabel('Density')
plt.legend()

# Subplot 2: CDF
plt.subplot(2, 1, 2)
plt.plot(x, cdf, label=f'Beta({u}, {v}) CDF')
plt.axhline(y=alpha, color='red', linestyle='--', label=f'{alpha}-th quantile (y={alpha})')
plt.axvline(x=quantile, color='blue', linestyle='--', label=f'Quantile value (x={quantile:.4f})')
plt.title('Beta Distribution CDF')
plt.ylabel('Cumulative Probability')
plt.legend()

plt.show()
